In [247]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler
from torchvision.models import resnet101

In [248]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)),
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [249]:
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [250]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
# Load the test dataset
test_dataset = datasets.CIFAR10(root='/kaggle/working/', train=False, transform=transform_test, download=True)

# Define the sizes for validation and test sets
val_size = 5000  # Number of samples for validation
test_size = len(test_set) - val_size

# Split the test dataset into validation and test datasets
val_set, test_set = random_split(test_set, [val_size, test_size])

val_loader = DataLoader(dataset=val_set, batch_size=128, shuffle=False)
test_loader = DataLoader(dataset=test_set, batch_size=128, shuffle=False)

print(f"Training dataset size: {len(train_set)}")
print(f"Validation dataset size: {len(val_set)}")
print(f"Test dataset size: {len(test_set)}")

Files already downloaded and verified
Training dataset size: 50000
Validation dataset size: 5000
Test dataset size: 5000


In [251]:
model = resnet101(pretrained=True)

In [252]:
model.fc.out_features

1000

In [254]:
model.conv1 = nn.Conv2d(
    in_channels=3,
    out_channels=64,
    kernel_size=(3, 3),
    stride=(1, 1),
    padding=(1, 1),
    bias=False  
)

model.fc = nn.Linear(model.fc.in_features, 10) 


In [255]:
model.conv1

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [256]:
model.fc

Linear(in_features=2048, out_features=10, bias=True)

In [257]:
# Training settings
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01,momentum=0.9 )
scheduler = lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.1)

True


In [258]:
# Training Loop with Validation Loss and Accuracy
num_epochs = 400

for epoch in range(num_epochs):
    # Training Phase
    model.train()  # Set the model to training mode
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update model parameters
        
        running_train_loss += loss.item()  # Accumulate training loss
        
        # Track training accuracy
        _, predicted = outputs.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()
    
    train_loss = running_train_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Validation Phase
    model.eval()  
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():  
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute validation loss
            running_val_loss += loss.item()
            
            # Track validation accuracy
            _, predicted = outputs.max(1)
            total_val += labels.size(0)
            correct_val += predicted.eq(labels).sum().item()
    
    val_loss = running_val_loss / len(test_loader)
    val_accuracy = 100 * correct_val / total_val

    # Print training and validation metrics
    print(f'Epoch [{epoch + 1}/{num_epochs}]')
    print(f'Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%')
    print(f'Val Loss: {val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%')
    
    scheduler.step()  

Epoch [1/400]
Train Loss: 1.1409 | Train Accuracy: 60.64%
Val Loss: 1.1967 | Val Accuracy: 73.66%
Epoch [2/400]
Train Loss: 0.5974 | Train Accuracy: 79.55%
Val Loss: 0.4639 | Val Accuracy: 83.72%
Epoch [3/400]
Train Loss: 0.4314 | Train Accuracy: 85.06%
Val Loss: 0.3968 | Val Accuracy: 86.08%
Epoch [4/400]
Train Loss: 0.3504 | Train Accuracy: 87.81%
Val Loss: 0.3773 | Val Accuracy: 86.86%
Epoch [5/400]
Train Loss: 0.3038 | Train Accuracy: 89.49%
Val Loss: 0.3615 | Val Accuracy: 87.98%
Epoch [6/400]
Train Loss: 0.2608 | Train Accuracy: 91.02%
Val Loss: 0.5404 | Val Accuracy: 86.86%
Epoch [7/400]
Train Loss: 0.2400 | Train Accuracy: 91.56%
Val Loss: 0.3108 | Val Accuracy: 89.74%
Epoch [8/400]
Train Loss: 0.2124 | Train Accuracy: 92.65%
Val Loss: 0.3274 | Val Accuracy: 89.16%
Epoch [9/400]
Train Loss: 0.1903 | Train Accuracy: 93.38%
Val Loss: 0.3120 | Val Accuracy: 89.58%
Epoch [10/400]
Train Loss: 0.1788 | Train Accuracy: 93.74%
Val Loss: 0.2834 | Val Accuracy: 91.20%
Epoch [11/400]
Trai

KeyboardInterrupt: 

In [259]:
def check_accuracy(loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device = device)
            y = y.to(device = device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    #model.train()
    return num_correct / num_samples

In [260]:
check_accuracy(test_loader,model)

tensor(0.9250, device='cuda:0')

In [262]:
        torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss,
        }, "model_checkpoint0925.pth")